In [1]:
import numpy
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, BatchNormalization, Activation
from keras.constraints import maxnorm
from keras.layers.convolutional import Conv2D, MaxPooling2D
from keras.utils import np_utils
from keras.datasets import cifar10

Using TensorFlow backend.


In [22]:
cifar10.load_data

<function keras.datasets.cifar10.load_data()>

In [2]:
seed = 21
numpy.random.seed(seed)

In [5]:
(X_train, y_train), (X_test, y_test) = cifar10.load_data()


- to normalize the data we can simply divide the image values by 255
- first need to make the data a float type

In [9]:
X_train = X_train.astype('float32')
X_test = X_test.astype('float32')
X_train = X_train / 255.0
X_test = X_test / 255.0

- We are effectively doing binary classification here because an image either belongs to one class or it doesn't, it can't fall somewhere in-between, so we use `to_categorical()` from `np_utils`

In [10]:
y_train = np_utils.to_categorical(y_train)
y_test = np_utils.to_categorical(y_test)
num_classes = y_test.shape[1]

- the number of channels/filters we want (that's the 32 below), the size of the filter we want (3 x 3 in this case).
- `relu` is the most common activation, and `padding='same'` just means we aren't changing the size of the image at all

In [23]:
model = Sequential()
model.add(Conv2D(32, (3, 3), input_shape=X_train.shape[1:], padding='same'))
model.add(Activation('relu'))

- here we need to dropout the overfitting values, which functions by randomly.
- `0.2` means it drops `20%` of the existing connections

In [24]:
model.add(Dropout(0.2))

In [25]:
# Batch Normalization normalizes the inputs heading into the next layer
model.add(BatchNormalization())

- Now comes another convolutional layer, but the filter size increases so the network can learn more complex representations.
- the pooling layer, as discussed before this helps make the image classifier more robust so it can learn relevant patterns

In [26]:
model.add(Conv2D(64, (3, 3), padding='same'))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.2))
model.add(BatchNormalization())

In [27]:
model.add(Conv2D(64, (3, 3), padding='same'))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.2))
model.add(BatchNormalization())

In [28]:
model.add(Conv2D(128, (3, 3), padding='same'))
model.add(Activation('relu'))
model.add(Dropout(0.2))
model.add(BatchNormalization())

In [29]:
model.add(Flatten())
model.add(Dropout(0.2))

In [30]:
model.add(Dense(256, kernel_constraint=maxnorm(3)))
model.add(Activation('relu'))
model.add(Dropout(0.2))
model.add(BatchNormalization())
model.add(Dense(128, kernel_constraint=maxnorm(3)))
model.add(Activation('relu'))
model.add(Dropout(0.2))
model.add(BatchNormalization())
model.add(Dense(num_classes))
model.add(Activation('softmax'))

In [31]:
epochs = 25
optimizer = 'Adam'
# compile the model with our chosen parameters and specify a metric to use
model.compile(loss='categorical_crossentropy', optimizer=optimizer, metrics=['accuracy'])

In [18]:
print(model.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 32, 32, 32)        896       
_________________________________________________________________
activation_1 (Activation)    (None, 32, 32, 32)        0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 32, 32, 32)        0         
_________________________________________________________________
batch_normalization_1 (Batch (None, 32, 32, 32)        128       
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 32, 32, 64)        18496     
_________________________________________________________________
activation_2 (Activation)    (None, 32, 32, 64)        0         
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 16, 16, 64)        0         
__________

- We will be training on `50000` samples and validating on `10000` samples.

In [ ]:
model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=epochs, batch_size=64)

In [ ]:
scores = model.evaluate(X_test, y_test, verbose=0)
print("Accuracy: %.2f%%" % (scores[1]*100))